<a href="https://colab.research.google.com/github/chemicoPy/MACD-RSI-STOCHASTIC-strategy/blob/US-(United-States)/stock_mkt_web_scraper_(US).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

In [ ]:
drive.mount("/content/gdrive")

In [ ]:
!pip install investpy
!pip install requests
!pip install tabulate

In [3]:
import numpy as np
from numpy import *
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
from tabulate import tabulate
import re
import requests
import os
import io
import time
from datetime import date, timedelta
from collections import deque
from matplotlib.pyplot import figure
import investpy
import warnings
warnings.filterwarnings('ignore')

In [4]:
""" To see US stock list: Just uncomment below line """
#US_stocks = investpy.stocks.get_stocks_list(country="United States")


tickers_list = ["msft","aapl","tsla","googl","amzn","fb","tsm","baba","nvda","v","jpm","jnj","wmt","unh",
               "bac","ma","hd","pg","dis","asml","pypl","adbe","cmcsa","nke", "lly", "AROW"]


In [ ]:
from dateutil.relativedelta import relativedelta
prev = date.today() - relativedelta(years=2)

from datetime import datetime
now = datetime.now()
move_from_date = now.strftime("%d/%m/%Y")
move_to_date = (date.today()- timedelta(days=26)).strftime("%d/%m/%Y")




In [16]:
print(move_from_date)
print(move_to_date)

16/10/2021
20/09/2021


In [14]:
allUrlData = {}
for ticker in tickers_list:
    allUrlData[ticker] = investpy.get_stock_historical_data(ticker,
                                        country='United States',
                                        from_date=str(move_from_date),
                                        to_date=str(move_to_date))

#data = pd.concat(allUrlData, ignore_index = False)
##print(data)
#data.to_csv("Yahoo Finance Data.csv")

##data.set_index("STOCKS",inplace = True)



ValueError: ignored

In [ ]:
for  k,v in allUrlData.items():
    allUrlData[k] = v.tail(26)
    
    """ MACD Calculation """
    
    exp1 = allUrlData[k]["Close"].ewm(span=12, adjust=False).mean()
    exp2 = allUrlData[k]["Close"].ewm(span=26, adjust=False).mean()
    allUrlData[k]["macd"] = exp1 - exp2

    """ MACD Buy & Sell signal: """

    signal_line_span = 9
    
    moving_avg = 'EMA'

    allUrlData[k].dropna(axis = 0, inplace = True)
   
    signal_line_col = str(signal_line_span) + '_' + moving_avg
    
    allUrlData[k][signal_line_col] = allUrlData[k]["macd"].ewm(span=signal_line_span, adjust=False).mean()
    
    MACD_conditions = [((allUrlData[k]["macd"]<-2) & (allUrlData[k][signal_line_col]<-2)) , ((allUrlData[k]["macd"]>0) & (allUrlData[k][signal_line_col]>0))]

    MACD_signals = ["Buy", "Sell"]

    allUrlData[k]['MACD_Signals'] = np.select(MACD_conditions, MACD_signals, default = "NIL")    
    
#    macd.plot(label='MACD', color='g')
#    ax = exp3.plot(label='Signal Line', color='r')
#    allUrlData[k]["close"].plot(ax=ax, secondary_y=True, label="AAPL")
    
    """Plots """
#    ax.set_ylabel('MACD')
#    ax.right_ax.set_ylabel('Price ($)')
#    ax.set_xlabel('Date')
#    lines = ax.get_lines() + ax.right_ax.get_lines()
#    ax.legend(lines, [l.get_label() for l in lines], loc='upper left')
    
    #plt.savefig(os.path.join(image_file_folder,"MACD Oscillator.png"))
#    plt.show()

    """ Stochastic Oscillator Calculation """
    
    allUrlData[k]['High'] = allUrlData[k]['High'].rolling(14).max()
    allUrlData[k]['Low'] = allUrlData[k]['Low'].rolling(14).min()
    allUrlData[k]['%K'] = (allUrlData[k]["Close"] - allUrlData[k]['Low'])*100/(allUrlData[k]['High'] - allUrlData[k]['Low'])
    allUrlData[k]['%D'] = allUrlData[k]['%K'].rolling(3).mean()

    """ Plots - upper and lower limits are indicated in the below plot """
#    ax = allUrlData[k][['%K', '%D']].plot()
#    allUrlData[k]["adjclose"].plot(ax=ax, secondary_y=True)
#    ax.axhline(20, linestyle='--', color="r")
#    ax.axhline(80, linestyle="--", color="r")
    #plt.savefig(os.path.join(image_file_folder,"Stochastic Oscillator.png"))
#    plt.show()

    """ Buy & Sell Signal for Stochastic """

    stoch_conditions = [((allUrlData[k]['%K']<20) & (allUrlData[k]['%D']<20)) , ((allUrlData[k]['%K']>80) & (allUrlData[k]['%D']>80))]

    stoch_signals = ["Buy", "Sell"]

    allUrlData[k]['Stochastic Signal'] = np.select(stoch_conditions, stoch_signals, default = "NIL")


    """ RSI (Relative Strength Index) Calculation """
    
    delta = allUrlData[k]["Close"].diff()
    up = delta.clip(lower=0)
    down = -1*delta.clip(upper=0)
    ema_up = up.ewm(com=13, adjust=False).mean()
    ema_down = down.ewm(com=13, adjust=False).mean()
    rs = ema_up/ema_down
    
    allUrlData[k]['RSI'] = 100 - (100/(1 + rs))
    
    """ Buy & Sell Signal for RSI """

    RSI_conditions = [(allUrlData[k]['RSI']>50) , (allUrlData[k]['RSI']<50)]

    RSI_signals = ["Buy", "Sell"]

    allUrlData[k]['RSI Signal'] = np.select(RSI_conditions, RSI_signals, default = "NIL")

    
    
    """ Plots """
#    fig, (ax1, ax2) = plt.subplots(2)
#    ax1.get_xaxis().set_visible(False)
#    fig.suptitle("AAPL")
#    
#    allUrlData[k]["close"].plot(ax=ax1)
#    ax1.set_ylabel('Price ($)')
#    data['RSI'].plot(ax=ax2)
#    ax2.set_ylim(0,100)
#    ax2.axhline(30, color='r', linestyle='--')
#    ax2.axhline(70, color='r', linestyle='--')
#    ax2.set_ylabel("AAPL")


#    allUrlData[k]['Position'] = allUrlData[k]['Signal'].diff()
    
#    allUrlData[k].rename(columns = {"Signal": "Buy/Sell signal"}, inplace = True)
    
    """ Cleaning - Dropping columns  """

    
    allUrlData[k].drop("Open",inplace = True, axis = 1)
    allUrlData[k].drop("High",inplace = True, axis = 1)
    allUrlData[k].drop("Low",inplace = True, axis = 1)
    allUrlData[k].drop("Close",inplace = True, axis = 1)
#    allUrlData[k].drop("adjclose",inplace = True, axis = 1)
#    allUrlData[k].drop("%D",inplace = True, axis = 1)
    allUrlData[k].drop("9_EMA",inplace = True, axis = 1)

#    allUrlData[k].set_index("ticker", inplace = True)
    
    allUrlData[k] = allUrlData[k].tail(1)
    
    
  
